# Brain Tumor Detection Using Parallel Processing

***Mhdtaha alfarawati - 60101364***

## Assignment Overview
In this assignment, you will be tasked with developing a machine learning model for detecting brain tumors from MRI images. You will leverage the power of parallel processing to efficiently handle the large dataset and speed up the computation-intensive tasks involved in image processing and model training.

## Dataset
The dataset consists of MRI images classified into two categories:
- `yes`: Images that contain brain tumors.
- `no`: Images that do not contain brain tumors.

Your goal is to preprocess these images using various filters and techniques, extract relevant features, then train a machine learning model to accurately classify the images as having a tumor or not.

## Parallel Processing
To optimize the performance of your image processing and model training, you are required to implement parallel processing techniques. This could involve using Python's `multiprocessing` and `threading` module to parallelize tasks such as image preprocessing, feature extraction (applying filters), or model training.

## Objectives
1. Load the MRI images using OpenCV.
2. Implement parallel processing to efficiently handle image processing and model training.
3. Train a machine learning model for brain tumor classification.
4. Evaluate the performance of your model on a test set.

## Submission
- Your submission should include the completed Jupyter Notebook with all the code for loading the data, preprocessing, parallel processing implementation, model training, and evaluation. 
- Additionally, provide a brief report discussing your approach, and explaining your code the results obtained.


# Part I: Guided Code (60%)
The following cells in this notebook will demonstrate a sequential example of the brain tumor detection process. This example includes steps such as data loading, pre-processing, feature extraction using methods like GLCM (Gray Level Co-occurrence Matrix) and LBP (Local Binary Patterns), and finally, classification. This sequential process serves as a baseline for what you are expected to parallelize.

## Your Task
After understanding the sequential processing steps, your task is to refactor the code to utilize multiprocessing or multithreading approaches, aiming to reduce the overall processing time. You should focus on parallelizing the most time-consuming tasks identified in the sequential example, such as image processing and feature extraction.

<span style="color: red;">**Remember**: the efficiency of your parallel processing implementation will be evaluated based on the reduction in processing time and the accuracy of your model.</span>

## Data Reading

In this section, we will load the MRI images from the dataset. The dataset consists of two folders: `yes` and `no`, representing images with and without brain tumors, respectively. We will use the `glob` module to list all the image files in these directories and then read them into memory for further processing.

### Creating a reading function

In [1]:
import glob
import cv2

def read_images(images_path):
    """
    Reads all images from a specified path using OpenCV.

    Parameters:
        - images_path (str): The path to the directory containing the images.
    Returns:
        - images (list): A list of images read from the directory.
    """
    images = []
    for file_path in images_path:
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
                images.append(image)
    return images

### Runing the reading function

In [2]:
# Define the path to the dataset
dataset_path = '../data/brain_tumor_dataset/'

# List all image files in the 'yes' and 'no' directories
yes_images = glob.glob(dataset_path + 'yes/*.jpg')
no_images = glob.glob(dataset_path + 'no/*.jpg')

yes_images = read_images(yes_images)
no_images = read_images(no_images)

print(f"Number of 'yes' images: {len(yes_images)}")
print(f"Number of 'no' images: {len(no_images)}")

Number of 'yes' images: 86
Number of 'no' images: 85


## Appyling filters to the images

In this section, we apply various filters to the images to enhance their features. The filters used are:

1. **Entropy Filter**: This filter measures the randomness in the image, highlighting regions with high information content (e.g., edges).

2. **Gaussian Filter**: This filter smooths the image by blurring it, reducing noise and details.

3. **Sobel Filter**: This edge-detection filter highlights the gradients (edges) in the image.

4. **Gabor Filter**: This filter is used for texture analysis, emphasizing edges and texture patterns.

5. **Hessian Filter**: This filter enhances blob-like structures in the image.

6. **Prewitt Filter**: Another edge-detection filter, similar to the Sobel filter, but with a different kernel.

The folowing code is how these filtres are applied to one image (<span style="color: red;">*Your job is to apply them to all images.*</span>).

### Code the applying the filters


In [3]:
from skimage.filters.rank import entropy
from skimage.morphology import disk
from scipy import ndimage as nd
from skimage.filters import sobel, gabor, hessian, prewitt
import matplotlib.pyplot as plt

In [4]:
image = yes_images[1]

# Apply filters
entropy_img = entropy(image, disk(2))
gaussian_img = nd.gaussian_filter(image, sigma=1)
sobel_img = sobel(image)
gabor_img = gabor(image, frequency=0.9)[1]
hessian_img = hessian(image, sigmas=range(1, 100, 1))
prewitt_img = prewitt(image)

# Store the original and filtered images in a dictionary
filtered_images = {
    'Original': image,
    'Entropy': entropy_img,
    'Gaussian': gaussian_img,
    'Sobel': sobel_img,
    'Gabor': gabor_img,
    'Hessian': hessian_img,
    'Prewitt': prewitt_img
}

### Displaying the results

In [5]:
# Display each filtered image
plt.figure(figsize=(18, 3))
for i, (filter_name, filtered_image) in enumerate(filtered_images.items()):
        plt.subplot(1, len(filtered_images), i + 1)
        plt.imshow(filtered_image, cmap='gray')
        plt.title(filter_name)
        plt.axis('off')
plt.show()

## <span style="color: blue;">Parallel Image Filtering</span>

In this part of the assignment, you will create a function for each filter and apply them in parallel to the images. You will store the results in dictionaries, similar to the example shown previously. Make sure to handle synchronization appropriately, as multiple threads or processes will access the images.

### <span style="color: green;">Tasks</span>
1. **Sequential execution:**
        1. Loop through the images in both lists: `yes_images` and `no_images` and apply the filters in parallel.
        2. For each image, create a dictionary containing the original and filtered images.
        3. Store these dictionaries in two lists: `yes_inputs` for images with tumors and `no_inputs` for images without tumors.
        4. Time the execution to compute the speed up and the efficiency later.
2. **Parallel execution:**
        1. Create a separate function for each filter and write to be executed in parallel using either multiprocessing or multithreading.
        2. Use a multiprocessing or multithreading (*whatever you wish, from what you have learned in this course*) to manage parallel execution of the filter functions on the images and or the concurrent application on multiple images at the same time.
        3. Implement synchronization mechanisms to ensure safe access to shared resources.
        4. Measure the execution time of the parallel processing to compare it with the sequential execution.
### <span style="color: red;">Warning</span>
- Be cautious about the concurrent access to images by multiple threads or processes. Use appropriate synchronization mechanisms to prevent race conditions and ensure data integrity.
- Carefully choose which parallelization paradigm you will use. *Efficiency* and *Speed* are of utmost importance. You need to see a positive impact on the speedup.
### <span style="color: green;">**Hint:**</span>
When you run you code for testing, run it only one the 4 or 5 first image. Only run on all images in the final version.

In [5]:
## The sequential version

import time
from tqdm import tqdm

def process_images(images):
    processed_images = []
    for image in tqdm(images):
        filtered_images = {
            'Original': image,
            'Entropy': entropy(image, disk(2)),
            'Gaussian': nd.gaussian_filter(image, sigma=1),
            'Sobel': sobel(image),
            'Gabor': gabor(image, frequency=0.9)[1],
            'Hessian': hessian(image, sigmas=range(1, 100, 1)),
            'Prewitt': prewitt(image)
        }
        processed_images.append(filtered_images)
    return processed_images

# Example usage
start_time = time.time()
yes_inputs = process_images(yes_images)
no_inputs = process_images(no_images)
end_time = time.time()

execution_time = end_time - start_time
print(f"Sequential execution time: {execution_time} seconds")


100%|██████████| 85/85 [42:44<00:00, 30.18s/it]  

Sequential execution time: 8314.084400177002 seconds


## The parallel version

#### Libraries

In [6]:
import time
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
from skimage.filters.rank import entropy
from skimage.morphology import disk
from scipy import ndimage as nd
from skimage.filters import sobel, gabor, hessian, prewitt
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage.morphology import disk
import numpy as np

#### Functions

In [7]:
def apply_entropy(image):
    return entropy(image, disk(2))

def apply_gaussian(image):
    return gaussian_filter(image, sigma=1)

def apply_sobel(image):
    return sobel(image)

def apply_gabor(image):
    return gabor(image, frequency=0.9)[1]

def apply_hessian(image):
    return hessian(image, sigmas=range(1, 100, 1))

def apply_prewitt(image):
    return prewitt(image)

#### Multiprocessing Technique

In [9]:
def process_image_parallel(image):
    with ProcessPoolExecutor() as executor:
        future_entropy = executor.submit(apply_entropy, image)
        future_gaussian = executor.submit(apply_gaussian, image)
        future_sobel = executor.submit(apply_sobel, image)
        future_gabor = executor.submit(apply_gabor, image)
        future_hessian = executor.submit(apply_hessian, image)
        future_prewitt = executor.submit(apply_prewitt, image)
        
        results = {
            'Original': image,
            'Entropy': future_entropy.result(),
            'Gaussian': future_gaussian.result(),
            'Sobel': future_sobel.result(),
            'Gabor': future_gabor.result(),
            'Hessian': future_hessian.result(),
            'Prewitt': future_prewitt.result()
        }
    return results

def process_images_parallel(images):
    processed_images = []
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_image_parallel, image) for image in images]
        for future in tqdm(as_completed(futures), total=len(futures)):
            processed_images.append(future.result())
    return processed_images

####  Execution and Timing

In [10]:
start_time = time.time()
yes_inputs_parallel = process_images_parallel(yes_images)
no_inputs_parallel = process_images_parallel(no_images)
end_time = time.time()

execution_time_parallel = end_time - start_time
print(f"Parallel execution time: {execution_time_parallel} seconds")


  0%|          | 0/86 [00:00<?, ?it/s]

100%|██████████| 85/85 [11:33<00:00,  8.16s/it]

Parallel execution time: 2496.662308216095 seconds


## Analsys:
- Explain you parallelization>
- Analyze the speedup and efficiency of the parallel execution. Discuss the results and any trade-offs encountered.


### **Parallelization Explanation**

In this study, we leveraged parallel computing techniques to significantly accelerate the process of applying multiple image processing filters to a set of images. The parallelization was achieved through the Python `concurrent.futures.ProcessPoolExecutor`, which facilitates the execution of computationally intensive tasks in parallel across multiple processors. This approach allows for the concurrent processing of different images as well as the simultaneous application of various filters to a single image.

The code structure involves two primary functions: `process_image_parallel`, which applies multiple filters in parallel to a single image, and `process_images_parallel`, which extends this parallel processing to a batch of images. By employing futures, placeholders for the results of asynchronous operations, we submit tasks without waiting for them to complete and later collect the outcomes. This non-blocking model enables the CPU to work on multiple tasks concurrently, which will utilize the available computational resources.

### **Speedup and Efficiency Analysis**

The performance results present a winning case for parallel execution:

- Sequential Execution Time: 2496 seconds
- Parallel Execution Time: 8314 seconds

The sequential processing, where tasks are executed one after another, took approximately 8314 seconds to complete the given workload. In contrast, the parallel approach, which distributes tasks across multiple CPU cores, significantly reduced the total execution time to about 2496 seconds.


Speedup: The speedup of a parallel program is defined as the ratio of the time taken to complete a task in the best sequential algorithm to the time taken in parallel. In this case, the speedup is 8314/2496 ≈ 3.33, This indicates that the parallel version is over 3.3 times faster than its sequential counterpart.

Efficiency:  Efficiency measures the effectiveness of parallelization, calculated as the speedup divided by the number of processors used. 
in our case it's 4 cores, which means 3.33/4 = 0.8325, This efficiency value indicates that the parallel execution utilized 83.25% of the theoretical maximum performance of the 4 cores. This value reflects the degree to which the hardware resources were effectively used in the parallel execution.

### **Trade-offs**
I really didn't counter any trade-offs, just the time difference and the consumbtion of more resources in the parallel process. 

#### **Notes**:
Because of the nature of this assignment I couldn't make it all the way in one time, since I had to run it for sequential and then for parallel which took about 4.5 hours, then I have to run the guided programming part which will also need the last parallel part.

Because of time management I couldn't do the whole dataset for part 2 and I did for 10 images in each category just for the code to work and test it out **(only for part 2)**

The correct way was to test everything and then doing the whole assignment in one shot

In [10]:
# Running the code again for the sake of part 2 (with smaller dataset)

import time
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
from skimage.filters.rank import entropy
from skimage.morphology import disk
from scipy import ndimage as nd
from skimage.filters import sobel, gabor, hessian, prewitt
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage.morphology import disk
import numpy as np

def apply_entropy(image):
    return entropy(image, disk(2))

def apply_gaussian(image):
    return gaussian_filter(image, sigma=1)

def apply_sobel(image):
    return sobel(image)

def apply_gabor(image):
    return gabor(image, frequency=0.9)[1]

def apply_hessian(image):
    return hessian(image, sigmas=range(1, 100, 1))

def apply_prewitt(image):
    return prewitt(image)

def process_image_parallel(image):
    with ProcessPoolExecutor() as executor:
        future_entropy = executor.submit(apply_entropy, image)
        future_gaussian = executor.submit(apply_gaussian, image)
        future_sobel = executor.submit(apply_sobel, image)
        future_gabor = executor.submit(apply_gabor, image)
        future_hessian = executor.submit(apply_hessian, image)
        future_prewitt = executor.submit(apply_prewitt, image)
        
        results = {
            'Original': image,
            'Entropy': future_entropy.result(),
            'Gaussian': future_gaussian.result(),
            'Sobel': future_sobel.result(),
            'Gabor': future_gabor.result(),
            'Hessian': future_hessian.result(),
            'Prewitt': future_prewitt.result()
        }
    return results

def process_images_parallel(images):
    processed_images = []
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_image_parallel, image) for image in images]
        for future in tqdm(as_completed(futures), total=len(futures)):
            processed_images.append(future.result())
    return processed_images


start_time = time.time()
yes_inputs = process_images_parallel(yes_images[:20]) # The tested data below is on 20 images 
no_inputs = process_images_parallel(no_images[:20])
end_time = time.time()

execution_time_parallel = end_time - start_time
print(f"Parallel execution time: {execution_time_parallel} seconds")


  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [05:02<00:00, 15.13s/it]

Parallel execution time: 1022.6557655334473 seconds


# Part II: Half-guided programming (30 %).
In this part, you'll create the machine learning dataset.


## Adapting the images for machine learning

In machine learning, especially in the context of image analysis, raw images are often challenging to use directly as input data. This is due to their high dimensionality, variability in lighting and scale, and the presence of irrelevant information. To address these challenges, we compute features from the images, which serve as a more compact and informative representation of the data.

Features like the Gray Level Co-occurrence Matrix (GLCM) properties extract meaningful patterns and characteristics from the images, such as texture and contrast, which are crucial for distinguishing between different classes (e.g., tumorous vs. healthy tissue). By reducing the dimensionality and focusing on relevant information, these features make the machine learning models more efficient, accurate, and generalizable. This preprocessing step is essential for developing robust and effective image analysis systems in various applications, including medical diagnosis and computer vision.


## GLCM Features and Their Formulas

1. **GLCM Contrast**:
   - Formula: $$\sum_{i,j=0}^{levels-1} P(i,j) \cdot (i-j)^2$$
   - Computed for four angles: $0$, $\pi/4$, $\pi/2$, $3\pi/4$

      ```python
      c = feature.graycoprops(graycom, 'contrast')
      ```

2. **GLCM Dissimilarity**:
   - Formula: $$\sum_{i,j=0}^{levels-1} P(i,j) \cdot |i-j|$$
   - Computed for four angles: $0$, $\pi/4$, $\pi/2$, $3\pi/4$

      ```python
      d = feature.graycoprops(graycom, 'dissimilarity')
      ```

3. **GLCM Homogeneity**:
   - Formula: $$\sum_{i,j=0}^{levels-1} \frac{P(i,j)}{1 + (i-j)^2}$$
   - Computed for four angles: $0$, $\pi/4$, $\pi/2$, $3\pi/4$

      ```python
      h = feature.graycoprops(graycom, 'homogeneity')
      ```

4. **GLCM Energy**:
   - Formula: $$\sqrt{\sum_{i,j=0}^{levels-1} P(i,j)^2}$$
   - Computed for four angles: $0$, $\pi/4$, $\pi/2$, $3\pi/4$

      ```python
      e = feature.graycoprops(graycom, 'energy')
      ```

5. **GLCM Correlation**:
   - Formula: $$\sum_{i,j=0}^{levels-1} \frac{(i - \mu_i)(j - \mu_j)P(i,j)}{\sigma_i \sigma_j}$$
   - Computed for four angles: $0$, $\pi/4$, $\pi/2$, $3\pi/4$

      ```python
      corr = feature.graycoprops(graycom, 'correlation')
      ```

6. **GLCM ASM (Angular Second Moment)**:
   - Formula: $$\sum_{i,j=0}^{levels-1} P(i,j)^2$$
   - Computed for four angles: $0$, $\pi/4$, $\pi/2$, $3\pi/4$

      ```python
      asm = feature.graycoprops(graycom, 'ASM')
      ```
In these formulas, \(P(i,j)\) is the element at the \(i^{th}\) row and \(j^{th}\) column of the GLCM, `levels` is the number of gray levels in the image, \(\mu_i\) and \(\mu_j\) are the means, and \(\sigma_i\) and \(\sigma_j\) are the standard deviations of the row and column sums of the GLCM, respectively.

## The code a exemple for feature extraction

In [11]:
## The Code exemple
import numpy as np
import pandas as pd
import skimage.feature as feature

# Function to compute GLCM features for an image
def compute_glcm_features(image, 
                                                    filter_name):
    """
    Computes GLCM (Gray Level Co-occurrence Matrix) features for an image.

    Parameters:
    - image: A 2D array representing the image. Should be in grayscale.
    - filter_name: A string representing the name of the filter applied to the image.

    Returns:
    - features: A dictionary containing the computed GLCM features. The keys are
        formatted as "{filter_name}_{feature_name}_{angle_index}", where "angle_index"
        corresponds to the index of the angle used for the GLCM calculation (1-based).
        The features include contrast, dissimilarity, homogeneity, energy, correlation,
        and ASM (Angular Second Moment) for each angle (0, π/4, π/2, 3π/4).

    Notes:
    - The image is first converted from float to uint8 format, as the graycomatrix
        function expects integer values.
    - The GLCM is computed using four angles (0, π/4, π/2, 3π/4) with a distance of 1.
    - The GLCM properties are computed and flattened into a 1D array to handle multiple
        angles. Each property value for each angle is stored as a separate key in the
        resulting dictionary.
    """
    # Convert the image from float to int
    image = (image * 255).astype(np.uint8)

    # Compute the GLCM
    graycom = feature.graycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

    # Compute GLCM properties
    features = {}
    for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']:
            values = feature.graycoprops(graycom, prop).flatten()
            for i, value in enumerate(values):
                    features[f'{filter_name}_{prop}_{i+1}'] = value
    return features


def process_images(images_list, tumor_presence):
    """
    Processes a list of images, applies all filters, computes GLCM features, and adds a "Tumor" key.

    Parameters:
    - images_list: A list of dictionaries, where each dictionary contains filtered images with keys
      representing the filter names.
    - tumor_presence: An integer (0 or 1) indicating the presence (1) or absence (0) of a tumor.

    Returns:
    - glcm_features_list: A list of dictionaries, where each dictionary contains the GLCM features for
      all filtered images of one original image and a "Tumor" key indicating the presence or absence
      of a tumor.

    Notes:
    - The function iterates over each image in the input list. For each image, it applies all filters
      and computes the GLCM features using the compute_glcm_features function.
    - The "Tumor" key is added to each dictionary to indicate whether the image is from the "yes" (tumor)
      or "no" (no tumor) list.
    - The resulting list of dictionaries can be used to create a pandas DataFrame for machine learning
      tasks.
    """
    # Apply all filters to each image and compute GLCM features
    glcm_features_list = []
    for filtered_images in images_list:
        glcm_features = {}
        for key, image in filtered_images.items():
            glcm_features.update(compute_glcm_features(image, key))
        glcm_features['Tumor'] = tumor_presence
        glcm_features_list.append(glcm_features)
    return glcm_features_list



# Process the 'yes' and 'no' image lists
yes_glcm_features = process_images(yes_inputs, 1)
no_glcm_features = process_images(no_inputs, 0)

# Combine the features into a single list
all_glcm_features = yes_glcm_features + no_glcm_features

# Convert the list of dictionaries to a pandas DataFrame
dataframe = pd.DataFrame(all_glcm_features)

# Print the first few rows of the DataFrame
print(dataframe.shape)

# Shuffle the DataFrame
shuffled_dataframe = dataframe.sample(frac=1).reset_index(drop=True)

# Print the first few rows of the shuffled DataFrame
print(shuffled_dataframe.head())

/home/student/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


(40, 169)
   Original_contrast_1  Original_contrast_2  Original_contrast_3  \
0          1501.916315          2117.519805          1630.910160   
1          1200.196190          1424.195910          1029.254325   
2          3511.752387          4194.167801          2977.833852   
3          2348.559699          2657.766256           805.484824   
4           838.241882          1117.650505           775.643355   

   Original_contrast_4  Original_dissimilarity_1  Original_dissimilarity_2  \
0          2085.296860                  8.873884                 12.514600   
1          1474.218112                 12.238095                 14.617227   
2          4160.902763                 21.113880                 25.122747   
3          2643.542159                 14.283679                 16.895111   
4          1132.102735                  6.086054                  8.081314   

   Original_dissimilarity_3  Original_dissimilarity_4  Original_homogeneity_1  \
0                  9.451261    

## Parallelization Instructions: What you need to do:

To get the grade, parallelize the given code:

1. **Identify Parallelizable Components**:
   - The `process_images` function is a prime candidate for parallelization. It processes each image independently, making it suitable for concurrent execution.

2. **Choose a Parallelization Method**:
   - You can use either multiprocessing or multithreading.
   - I chose **multiprocessing**

3. **Modify the `process_images` Function**:
   - Use a `multiprocessing`/`threading` to parallelize the processing of images. Replace the for loop with a call the appropriate parallel control algorithm of your choice to  `compute_glcm_features` to each image in parallel.

4. **Handle Synchronization**:
   - Ensure that shared resources are properly synchronized. In this case, the individual image processing tasks are independent, so synchronization is not a major concern. However, be cautious when aggregating results or writing to shared data structures.

5. **Measure Performance**:
   - Compare the execution time of the parallelized version with the original version. Use the `time` module to measure the start and end times of the `process_images` function.

6. **Optimize**:
   - Experiment with different numbers of processes in the multiprocessing pool to find the optimal setting for your system.

Example code snippet for parallelization using multiprocessing:

#### Parellel

In [13]:
import numpy as np
import pandas as pd
import skimage.feature as feature
from concurrent.futures import ProcessPoolExecutor
import time

# Function to compute GLCM features for an image
def compute_glcm_features(image, filter_name):
    image = (image * 255).astype(np.uint8)
    graycom = feature.graycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    features = {}
    for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']:
            values = feature.graycoprops(graycom, prop).flatten()
            for i, value in enumerate(values):
                    features[f'{filter_name}_{prop}_{i+1}'] = value
    return features

def process_image_task(args):
    filtered_images, tumor_presence = args
    glcm_features = {}
    for key, image in filtered_images.items():
        glcm_features.update(compute_glcm_features(image, key))
    glcm_features['Tumor'] = tumor_presence
    return glcm_features

def process_images_parallel(images_list, tumor_presence):
    args = [(filtered_images, tumor_presence) for filtered_images in images_list]
    with ProcessPoolExecutor() as executor:
        glcm_features_list = list(executor.map(process_image_task, args))
    return glcm_features_list

# Timing the parallelized version
start_time_parallel = time.time()

yes_glcm_features_parallel = process_images_parallel(yes_inputs, 1)
no_glcm_features_parallel = process_images_parallel(no_inputs, 0)

all_glcm_features_parallel = yes_glcm_features_parallel + no_glcm_features_parallel
dataframe_parallel = pd.DataFrame(all_glcm_features_parallel)

# Shuffle the dataFrame again
shuffled_dataframe = dataframe.sample(frac=1).reset_index(drop=True)

end_time_parallel = time.time()

execution_time_parallel = end_time_parallel - start_time_parallel

print(f"Execution Time: {execution_time_parallel}")
print(dataframe_parallel.head())


Execution Time: 2.8138508796691895
   Original_contrast_1  Original_contrast_2  Original_contrast_3  \
0          2819.848921          3253.016747          2459.775936   
1           346.572451           436.877932           163.906056   
2           759.449677           945.078950           652.709015   
3           640.944769           740.318375           517.756410   
4          1787.663834          1923.764472          1749.946695   

   Original_contrast_4  Original_dissimilarity_1  Original_dissimilarity_2  \
0          3276.558945                 19.064768                 22.319017   
1           466.784749                  9.200596                 11.032638   
2           940.512505                  4.972055                  6.321946   
3           736.547913                  4.416252                  5.399598   
4          1921.136581                  8.767844                  9.817843   

   Original_dissimilarity_3  Original_dissimilarity_4  Original_homogeneity_1  \
0     

#### Sequential 
with time calculation to check the difference

In [14]:
import numpy as np
import pandas as pd
import skimage.feature as feature
import time

# Function to compute GLCM features for an image remains unchanged

def compute_glcm_features(image, filter_name):
    # Convert the image from float to int
    image = (image * 255).astype(np.uint8)

    # Compute the GLCM
    graycom = feature.graycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

    # Compute GLCM properties
    features = {}
    for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']:
            values = feature.graycoprops(graycom, prop).flatten()
            for i, value in enumerate(values):
                    features[f'{filter_name}_{prop}_{i+1}'] = value
    return features

# process_images function remains unchanged

def process_images(images_list, tumor_presence):
    glcm_features_list = []
    for filtered_images in images_list:
        glcm_features = {}
        for key, image in filtered_images.items():
            glcm_features.update(compute_glcm_features(image, key))
        glcm_features['Tumor'] = tumor_presence
        glcm_features_list.append(glcm_features)
    return glcm_features_list

# Main code to process images and calculate execution time

# Start timing
start_time = time.time()

# Process the 'yes' and 'no' image lists
yes_glcm_features = process_images(yes_inputs, 1)
no_glcm_features = process_images(no_inputs, 0)

# Combine the features into a single list
all_glcm_features = yes_glcm_features + no_glcm_features

# Convert the list of dictionaries to a pandas DataFrame
dataframe = pd.DataFrame(all_glcm_features)

shuffled_dataframe = dataframe.sample(frac=1).reset_index(drop=True)

end_time = time.time()
execution_time = end_time - start_time

# Print the first few rows of the DataFrame and the execution time
print(dataframe.shape)
print(f"Execution time: {execution_time} seconds")
print(shuffled_dataframe.head())


(40, 169)
Execution time: 8.59196138381958 seconds
   Original_contrast_1  Original_contrast_2  Original_contrast_3  \
0          1787.663834          1923.764472          1749.946695   
1          2819.848921          3253.016747          2459.775936   
2          1450.931465          1909.501073          1254.386384   
3           223.600606           320.683910           133.169098   
4           206.624178           374.873914           169.573564   

   Original_contrast_4  Original_dissimilarity_1  Original_dissimilarity_2  \
0          1921.136581                  8.767844                  9.817843   
1          3276.558945                 19.064768                 22.319017   
2          1898.970375                  9.165193                 12.160967   
3           342.393173                  7.627436                  9.837069   
4           341.945889                  7.401303                 10.167747   

   Original_dissimilarity_3  Original_dissimilarity_4  Original_homogen

# Part III: Non guided machine learning application (10 %)

## Training and Validating a Machine Learning Algorithm


1. **Split the Data**:
   - Divide the `shuffled_dataframe` into features (X) and target (y) variables.
   - Further split the data into training (75%) and testing (25%) sets using `train_test_split` from `sklearn.model_selection`.

2. **Choose three Models**:
   - Select three machine learning models to use for classification. For example, you can use a Random Forest Classifier, Support Vector Machine...

3. **Train the Model**:
   - Fit the model to the training data.
   - Parallelize the training of the models.

4. **Validate the Model**:
   - Use the trained model to make predictions on the test data.
   - Evaluate the model's performance using the confusion matrix and the metrics such as accuracy, precision, recall, and F1-score.

5. **Fine-Tune the Model** (Optional):
   - If necessary, adjust the model's hyperparameters and repeat the training and validation process to improve performance, using parallel programming.

Splitting the Data

In [15]:
from sklearn.model_selection import train_test_split

# Assuming 'Tumor' is the target variable
X = shuffled_dataframe.drop('Tumor', axis=1)
y = shuffled_dataframe['Tumor']

# Split the dataset into training and testing sets (75% training, 25% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


Choosing Three Models

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Instantiating the models
model_rf = RandomForestClassifier(random_state=42)
model_svc = SVC(random_state=42)
model_knn = KNeighborsClassifier()

Training the model

In [17]:
# List of models for easy iteration
models = [model_rf, model_svc, model_knn]
model_names = ['Random Forest', 'SVC', 'KNN']

# Dictionary to store models' results
model_results = {}

for model, name in zip(models, model_names):
    model.fit(X_train, y_train)  # Training the model
    model_results[name] = model  # Storing the trained model


Validating the Models

In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# Function to calculate metrics
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='binary')
    recall = recall_score(y_test, predictions, average='binary')
    f1 = f1_score(y_test, predictions, average='binary')
    return accuracy, precision, recall, f1

# Evaluating each model and printing the results
for name, model in model_results.items():
    accuracy, precision, recall, f1 = evaluate_model(model, X_test, y_test)
    print(f"{name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f9554daa840>
Traceback (most recent call last):
  File "/home/student/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/student/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/student/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/home/student/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no 

Random Forest - Accuracy: 0.7000, Precision: 0.7500, Recall: 0.8571, F1-Score: 0.8000
SVC - Accuracy: 0.6000, Precision: 0.8000, Recall: 0.5714, F1-Score: 0.6667
KNN - Accuracy: 0.5000, Precision: 0.6667, Recall: 0.5714, F1-Score: 0.6154


Fine-Tuning the Models

In [19]:
from sklearn.model_selection import GridSearchCV
# For RandomForestClassifier
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
}

grid_search_rf = GridSearchCV(model_rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# Best parameters and score for RandomForestClassifier
print(f"Best parameters (Random Forest): {grid_search_rf.best_params_}")
print(f"Best score (Random Forest): {grid_search_rf.best_score_}")


/home/student/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/student/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/student/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/student/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Best parameters (Random Forest): {'max_depth': None, 'n_estimators': 200}
Best score (Random Forest): 0.7333333333333332
